In [ ]:
P = 2 ** 255 - 19
A24 = 121665
def bytes_to_int(bytes):

    result = 0

    for b in bytes:
        result = result * 256 + int(b)

    return result


def int_to_bytes(value, length):
    result = []
    for i in range(0, length):
        result.append(value >> (i * 8) & 0xff)

    return result

def cswap(swap, x_2, x_3):
    dummy = swap * ((x_2 - x_3) % P)
    x_2 = x_2 - dummy
    x_2 %= P
    x_3 = x_3 + dummy
    x_3 %= P
    return (x_2, x_3)

# Based on https://tools.ietf.org/html/rfc7748
def X25519(k, u):
    x_1 = u
    x_2 = 1
    z_2 = 0
    x_3 = u
    z_3 = 1
    swap = 0

    for t in reversed(range(255)):
        k_t = (k >> t) & 1
        swap ^= k_t
        x_2, x_3 = cswap(swap, x_2, x_3)
        z_2, z_3 = cswap(swap, z_2, z_3)
        swap = k_t

        A = x_2 + z_2
        A %= P

        AA = A * A
        AA %= P

        B = x_2 - z_2
        B %= P

        BB = B * B
        BB %= P

        E = AA - BB
        E %= P

        C = x_3 + z_3
        C %= P

        D = x_3 - z_3
        D %= P

        DA = D * A
        DA %= P

        CB = C * B
        CB %= P

        x_3 = ((DA + CB) % P)**2
        x_3 %= P

        z_3 = x_1 * (((DA - CB) % P)**2) % P
        z_3 %= P

        x_2 = AA * BB
        x_2 %= P

        z_2 = E * ((AA + (A24 * E) % P) % P)
        z_2 %= P

    x_2, x_3 = cswap(swap, x_2, x_3)
    z_2, z_3 = cswap(swap, z_2, z_3)

    return (x_2 * pow(z_2, P - 2, P)) % P

def decodeScalar25519(k):
  k_list = [(b) for b in k]
  k_list[0] &= 248
  k_list[31] &= 127
  k_list[31] |= 64
  return decodeLittleEndian(k_list)

def decodeLittleEndian(b):
    return sum([b[i] << 8*i for i in range( 32 )])

def unpack2(s):
    if len(s) != 32:
        raise ValueError('Invalid Curve25519 scalar (len=%d)' % len(s))
    t = sum((ord(s[i]) ) << (8 * i) for i in range(31))
    t += (((ord(s[31]) ) & 0x7f) << 248)
    return t

def pack(n):
    return ''.join([chr((n >> (8 * i)) & 255) for i in range(32)])

def clamp(n):
    n &= ~7
    n &= ~(128 << 8 * 31)
    n |= 64 << 8 * 31
    return n

# Return nP
def multscalar(n, p):
    n = clamp(decodeScalar25519(n))
    p = unpack2(p)
    return pack(X25519(n, p))

# Start at x=9. Find point n times x-point
def base_point_mult(n):
    n = clamp(decodeScalar25519(n))
    return pack(X25519(n, 9))

In [ ]:
import os
import binascii


class ECDiffieHellman:

  private_key = None
  generated_public = None
  shared_secret = None

  def __init__(self):
    self.private_key = os.urandom(32)

  def print_private(self):
    print (bytes_to_int(self.private_key))

  def compute_public(self):
    self.generated_public = base_point_mult(self.private_key)

  def print_public(self):
    if self.generated_public == None:
      self.compute_public()
    print (binascii.hexlify(self.generated_public.encode()))

  def compute_shared_secret(self, pub):
    self.shared_secret = multscalar(self.private_key, pub)

  def print_shared_secret(self):
    if self.shared_secret == None:
      print('Not computed yet!')
      pass
    print(binascii.hexlify(self.shared_secret.encode()))



In [ ]:
pip install pycryptodome

In [ ]:
from Crypto.Util.number import getPrime
import time

# Choose a 255-bit prime for P
BITLENGTH = 3072
start = time.time()
prime_3072 = getPrime(BITLENGTH)
end = time.time()

print(prime_3072)

print('time elapsed : ', end - start)


3785114712848260749569752031791836760549317394049388572494041695173789897026089755114479912871685593292562953368882592468268112126456224665991848705546737726122601007390987168896695716335902588895933402109287518059622521451225765299764270862999512014329599558609638760867859255384371646430066769205437506432808565427460870237857411147618889369879751379325041737603684644904334209905587274143600083889081315607459939577201707125826414723910525136633553523506910825988520942779417355496632527460213423391813025200432821064061994496648866135116661375535594492470473543919959465850431078583391263210269319645735088795678593880233763585534788332728237350628475379232519328130244731632369835270239161594889240274822074538219837637236913461006406434510343337223424397529063066544918515416753955768170820264580874331046242683817274433194111103371976448075296487070507185610176899545450855812659813347021637138158619520688333417238257
time elapsed :  25.807652950286865


In [ ]:
def power(x, y, p) :
    res = 1     # Initialize result

    # Update x if it is more
    # than or equal to p

    x = x % p

    if (x == 0) :
        return 0

    while (y > 0) :

        # If y is odd, multiply
        # x with result
        if ((y & 1) == 1) :
            res = (res * x) % p

        # y must be even now
        y = y >> 1      # y = y/2
        x = (x * x) % p

    return res

In [ ]:
import random


class DiffieHellman:

  P = prime_3072
  generator = 2
  private_key = None
  generated_public = None
  shared_secret = None

  def __init__(self):
    self.private_key = random.getrandbits(BITLENGTH - 1)

  def print_private(self):
    print(self.private_key)

  def compute_public(self):
    self.generated_public = pow(self.generator, self.private_key, self.P)

  def print_public(self):
    if self.generated_public == None:
      self.compute_public()
    print(self.generated_public)

  def compute_shared_secret(self, pub):
    self.shared_secret = pow(pub, self.private_key, self.P)

  def print_shared_secret(self):
    if self.shared_secret == None:
      print('Not computed yet!')
      pass
    print (self.shared_secret)

In [ ]:
import time

print('ECDH :\n\n')

bob = ECDiffieHellman()
alice = ECDiffieHellman()

print('\nAlice private (a):')
alice.print_private()
print ('\nBob private (b):')
bob.print_private()
print('')

bob.compute_public()        # b G
alice.compute_public()      # a G


print ('\nAlice public (aG)')
alice.print_public()
print ('\nBob public (bG)')
bob.print_public()
print('')

start = time.time()
bob.compute_shared_secret(alice.generated_public) # b (aG)
end = time.time()
alice.compute_shared_secret(bob.generated_public) # a (bG)

print ('\nAlice shared secret (aG)')
alice.print_shared_secret()
print ('\nBob shared secret (bG)')
bob.print_shared_secret()

print('time elapsed : ', end - start)

print('\n\n############################################################################################')
print('\nDiffie-Hellman :\n\n')

bob = DiffieHellman()
alice = DiffieHellman()

print('\nAlice private (a):')
alice.print_private()
print ('\nBob private (b):')
bob.print_private()
print('')

bob.compute_public()        # G^b
alice.compute_public()      # G^a


print ('\nAlice public (G^a)')
alice.print_public()
print ('\nBob public (G^b)')
bob.print_public()
print('')

start = time.time()
bob.compute_shared_secret(alice.generated_public) # (G^a)^b
end = time.time()
alice.compute_shared_secret(bob.generated_public) # (G^b)^a

print('time elapsed : ', end - start)

print ('\nAlice shared secret (G^a)^b')
alice.print_shared_secret()
print ('\nBob shared secret (G^a)^b')
bob.print_shared_secret()


ECDH :



Alice private (a):
11100877893498759004922910304626501348876429505662346118012913122201431246192

Bob private (b):
54913557066508722201476058885443377536460715412996139811807596081437925632360


Alice public (aG)
b'c3aac3acc29a0cc2bc1fc3ab341b22021c62682772c383c3b17dc3b8c381c3adc3a32f68c3b82fc3a2c38339c2b676'

Bob public (bG)
b'c2a228c2ab2958c3bbc399c386c3b2665f6f277ec298601c1ec2b73ac2ae50c3b46ac299c2ba66c3b5c3b95bc3ad6d'


Alice shared secret (aG)
b'c2aac29950c297c280c3a65428c29403c298c2b3c2abc29c7957c3bb3cc39a77c28f2638c28942c2b639c2906d7cc2bc6a'

Bob shared secret (bG)
b'c2aac29950c297c280c3a65428c29403c298c2b3c2abc29c7957c3bb3cc39a77c28f2638c28942c2b639c2906d7cc2bc6a'
time elapsed :  0.0038144588470458984


############################################################################################

Diffie-Hellman :



Alice private (a):
2302456224455992053039035431082399272778576322749019902273479428164998191225272027143224785181642948554531168345513042109725750710927822

In [ ]:
execution_time_DH = []
execution_time_ECDH = []

for i in range(10):
  prime_3072 = getPrime(BITLENGTH)
  bob = ECDiffieHellman()
  alice = ECDiffieHellman()

  bob.compute_public()
  alice.compute_public()

  start = time.time()
  bob.compute_shared_secret(alice.generated_public)
  end = time.time()

  execution_time_ECDH.append(end - start)

  bob = DiffieHellman()
  alice = DiffieHellman()

  bob.compute_public()
  alice.compute_public()

  start = time.time()
  bob.compute_shared_secret(alice.generated_public)
  end = time.time()

  execution_time_DH.append(end - start)





In [ ]:
#print(execution_time_DH)
#print(execution_time_ECDH)

scale = []

for i in range(len(execution_time_DH)):
  scale.append(execution_time_DH[i] / execution_time_ECDH[i])

#print(scale)

In [ ]:
from tabulate import tabulate
table = []
table.append(['Iteration', 'DH execution_time', 'ECDH execution_time', 'DH / ECDH'])
for i in range(len(execution_time_DH)):
  table.append([i + 1, '{:.4f}'.format(execution_time_DH[i]), '{:.4f}'.format(execution_time_ECDH[i]), '{:.0f}'.format(scale[i])])
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

╒═════════════╤═════════════════════╤═══════════════════════╤═════════════╕
│   Iteration │   DH execution_time │   ECDH execution_time │   DH / ECDH │
╞═════════════╪═════════════════════╪═══════════════════════╪═════════════╡
│           1 │              0.0972 │                0.0024 │          41 │
├─────────────┼─────────────────────┼───────────────────────┼─────────────┤
│           2 │              0.0972 │                0.0025 │          39 │
├─────────────┼─────────────────────┼───────────────────────┼─────────────┤
│           3 │              0.1029 │                0.0026 │          39 │
├─────────────┼─────────────────────┼───────────────────────┼─────────────┤
│           4 │              0.1218 │                0.0039 │          31 │
├─────────────┼─────────────────────┼───────────────────────┼─────────────┤
│           5 │              0.099  │                0.0026 │          38 │
├─────────────┼─────────────────────┼───────────────────────┼─────────────┤
│           